<h2 align=center> Facial Expression Recognition with Keras</h2>

<a href="https://colab.research.google.com/github/PradyumnaKrishna/FER/blob/main/Model%20Training/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
from livelossplot import PlotLossesKerasTF
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

### Download Dataset and Checking it

In [ ]:
! wget https://github.com/PradyumnaKrishna/FER/raw/main/Model%20Training/fer2013.zip
! unzip fer2013.zip

In [ ]:
for expression in os.listdir("train/"):
    print(str(len(os.listdir("train/" + expression))) + " " + expression + " images")

### Generate Training and Validation Batches

In [ ]:
img_size = 48
batch_size = 32

datagen_train = ImageDataGenerator(rotation_range=15,
                                   width_shift_range=0.15,
                                   height_shift_range=0.15,
                                   shear_range=0.15,
                                   zoom_range=0.15,
                                   horizontal_flip=True)

train_generator = datagen_train.flow_from_directory("train/",
                                                   target_size=(img_size, img_size),
                                                   color_mode='grayscale',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=True)

datagen_validation = ImageDataGenerator(horizontal_flip=True)

validation_generator = datagen_train.flow_from_directory("test/",
                                                   target_size=(img_size, img_size),
                                                   color_mode='grayscale',
                                                   batch_size=batch_size,
                                                   class_mode='categorical',
                                                   shuffle=False)

### Create CNN Model

In [ ]:
# Initialising the CNN
model = Sequential()

# Convolution Block 1
model.add(Conv2D(64,(5,5), padding='same', activation='elu', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Conv2D(64,(5,5), padding='same', activation='elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Convolution Block 2
model.add(Conv2D(128,(3,3), padding='same', activation='elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), padding='same', activation='elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Convolution Block 3
model.add(Conv2D(256,(3,3), padding='same', activation='elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3), padding='same', activation='elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Flattening
model.add(Flatten())

# Fully Connected layer
model.add(Dense(128, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

# Output layer
model.add(Dense(7, activation='softmax'))

# Model Compilation
opt = Adam(0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Define Callbacks


In [ ]:
# Callback to save Model
checkpoint = ModelCheckpoint(
    "model_weights.h5",
    monitor='val_accuracy',
    save_weights_only=True,
    mode='max',
    verbose=1
)

# Callback to early stopping of Training
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

# Callback to reduce Learning Rate
lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-8,
    verbose=1,
)

callbacks = [
    PlotLossesKerasTF(),
    early_stopping,
    lr_scheduler,
    checkpoint,
]

### Train and Evaluate Model

In [ ]:
epochs = 200

steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size


history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = validation_steps,
    callbacks=callbacks
)

### Save Model as JSON string

In [ ]:
model_json = model.to_json()
with open("model.json", 'w') as json_file:
    json_file.write(model_json)